In [60]:
import h5py
import numpy as np
import pandas as pd

In [2]:
path = 'brick_kiln_v1.0'

examples_0.hdf5   examples_2.hdf5   examples_4.hdf5   examples_6.hdf5
examples_10.hdf5  examples_30.hdf5  examples_50.hdf5  examples_70.hdf5
examples_11.hdf5  examples_31.hdf5  examples_51.hdf5  examples_71.hdf5
examples_12.hdf5  examples_32.hdf5  examples_52.hdf5  examples_72.hdf5
examples_13.hdf5  examples_33.hdf5  examples_53.hdf5  examples_73.hdf5
examples_14.hdf5  examples_34.hdf5  examples_54.hdf5  examples_74.hdf5
examples_15.hdf5  examples_35.hdf5  examples_55.hdf5  examples_75.hdf5
examples_16.hdf5  examples_36.hdf5  examples_56.hdf5  examples_76.hdf5
examples_17.hdf5  examples_37.hdf5  examples_57.hdf5  examples_77.hdf5
examples_18.hdf5  examples_38.hdf5  examples_58.hdf5  examples_78.hdf5
examples_19.hdf5  examples_39.hdf5  examples_59.hdf5  examples_79.hdf5
examples_1.hdf5   examples_3.hdf5   examples_5.hdf5   examples_7.hdf5
examples_20.hdf5  examples_40.hdf5  examples_60.hdf5  examples_80.hdf5
examples_21.hdf5  examples_41.hdf5  examples_61.hdf5  examples_81.hdf5
examples

In [206]:
labels = np.array([]).reshape(0, 1)
indices = np.array([]).reshape(0,3)
bounds = np.array([]).reshape(0,4)
for i in range(87):
    with h5py.File(path + f'/examples_{i}.hdf5', 'r') as data:
        labels = pd.DataFrame(data=data['labels'], columns=['y']).astype(int)
        bounds = pd.DataFrame(data=data['bounds'], columns=['lon_top_left', 'lat_top_left', 'lon_bottom_right', 'lat_bottom_right'])
        indices = pd.DataFrame(data=data['indices'], columns=['indice_offset_num', 'indice_x', 'indice_y'])
        indexes = pd.DataFrame(data=range(labels.shape[0]), columns=['hdf5_idx'])
        if i == 0:
            df = pd.concat([labels, indexes, bounds, indices], axis=1)
            df.insert(1, 'hdf5_file', i)
        else:
            curr_df = pd.concat([labels, indexes, bounds, indices], axis=1)
            curr_df.insert(1, 'hdf5_file', i)
            df = pd.concat([df, curr_df])
            
df = df.reset_index(drop=True)

In [207]:
train = df.reset_index(                  # need to keep the index as a column
    ).groupby('y'                        # split by "group"
    ).apply(lambda x: x.sample(frac=0.8, random_state=42) # in each group, do the random split
    ).reset_index(drop=True              # index now is group id - reset it
    ).set_index('index')                 # reset the original index
valtest = df.drop(train.index)              # now we can subtract it from the rest of data

In [208]:
train = train.reset_index(drop=True)
valtest = valtest.reset_index(drop=True)

In [209]:
val = valtest.reset_index(                  # need to keep the index as a column
    ).groupby('y'                        # split by "group"
    ).apply(lambda x: x.sample(frac=0.5, random_state=42) # in each group, do the random split
    ).reset_index(drop=True              # index now is group id - reset it
    ).set_index('index')                 # reset the original index
test = valtest.drop(val.index)              # now we can subtract it from the rest of data

In [210]:
train.insert(1, 'partition', 0)
val.insert(1, 'partition', 1)
test.insert(1, 'partition', 2)

In [211]:
combined_df = pd.concat([train, val, test]).reset_index(drop=True)

In [213]:
combined_df.to_csv('list_eval_partition.csv', index=False)

In [212]:
combined_df

,y,partition,hdf5_file,hdf5_idx,lon_top_left,lat_top_left,lon_bottom_right,lat_bottom_right,indice_offset_num,indice_x,indice_y
0,0,0,85,210,91.971495,21.173291,91.977244,21.179040,947.0,687.0,0.0
1,0,0,17,421,88.625450,25.416214,88.631200,25.421963,209.0,105.0,0.0
2,0,0,55,156,88.918660,22.668088,88.924410,22.673837,687.0,156.0,0.0
3,0,0,46,926,89.470585,23.478728,89.476335,23.484477,546.0,252.0,0.0
4,0,0,14,503,91.132109,25.450709,91.137858,25.456459,203.0,541.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
86523,0,2,86,515,92.488925,20.828338,92.494674,20.834087,1007.0,777.0,0.0
86524,0,2,86,523,92.442931,20.822589,92.448680,20.828338,1008.0,769.0,0.0
86525,0,2,86,567,92.264705,20.782344,92.270454,20.788094,1015.0,738.0,0.0
86526,0,2,86,598,92.471677,20.753598,92.477426,20.759348,1020.0,774.0,0.0
